In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)
from sklearn.metrics import auc,roc_curve,roc_auc_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import copy

In [3]:
def auc(y,pred):
    fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
    score = metrics.auc(fpr, tpr)
    return score

In [4]:
# load data

A_train = pd.read_csv('data/A_train.csv')
B_train = pd.read_csv('data/B_train.csv')
B_test = pd.read_csv('data/B_test.csv')
NO = B_test['no']
Bcolumns = B_train.columns
B_train.drop('no',axis=1,inplace=True)
B_test.drop('no',axis=1,inplace=True)

In [5]:
# 统计特征中空值百分比

# 针对A_train
A_train_columns = A_train.columns
A_null_count_less = []
A_null_count_large = []
# threshold = 0.4
for i in A_train_columns:
    if ((A_train[i].isnull().sum()) / len(A_train[i]) <= 0.4):
        A_null_count_less.append([i,(A_train[i].isnull().sum()) / len(A_train[i])])
    else:
        A_null_count_large.append([i,(A_train[i].isnull().sum()) / len(A_train[i])])
# len(A_null_count_less)    335
# len(A_null_count_large)   156

# 针对B_train
B_train_columns = B_train.columns
B_null_count_less = []
B_null_count_large = []
# threshold = 0.63
for i in B_train_columns:
    if ((B_train[i].isnull().sum()) / len(B_train[i]) <= 0.63):
        B_null_count_less.append([i,(B_train[i].isnull().sum()) / len(B_train[i])])
    else:
        B_null_count_large.append([i,(B_train[i].isnull().sum()) / len(B_train[i])])

# len(B_null_count_less) 327
# len(B_null_count_large) 164

# 针对 B_test
B_test_columns = B_test.columns
B_test_count_less = []
B_test_count_large = []

for i in B_test_columns:
    if ((B_test[i].isnull().sum()) / len(B_test[i]) <= 0.63):
        B_test_count_less.append([i,(B_test[i].isnull().sum()) / len(B_test[i])])
    else:
        B_test_count_large.append([i,(B_test[i].isnull().sum()) / len(B_test[i])])

# len(B_test_count_large) 164
# len(B_test_count_less) 326



In [6]:
# 提取百分比大于47的数据

A_feature = pd.DataFrame(B_null_count_less).values[:,0]
B_feature = pd.DataFrame(B_null_count_less).values[:,0]
BT_feature = pd.DataFrame(B_test_count_less).values[:,0]

a_data = A_train[A_feature]
b_data = B_train[B_feature]
bt_data = B_train[BT_feature]

a_columns = a_data.columns
a_columns = a_columns.sort_values()

b_columns = b_data.columns   ## B_train columns，多了一个flag
b_columns = b_columns.sort_values()

bt_columns = bt_data.columns   ## B_test columns
bt_columns = bt_columns.sort_values()

a_data = A_train[a_columns]
b_data = B_train[b_columns]
bt_data = B_test[bt_columns]

## GBDT特征筛选+XGB训练模型-B数据集-AUC0.583

In [7]:
target_bb = b_data['flag']
b_data.drop('flag',axis=1,inplace=True)

bb_data = b_data.fillna(0)
bbt_data = bt_data.fillna(0)


# GBDT进行特征筛选

from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation
X_train, X_test,y_train,y_test = cross_validation.train_test_split(bb_data, target_bb, test_size=0.3, random_state=0)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1, max_depth=1, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
y_pred = clf.predict(X_test) # 返回预测标签
y_pro = clf.predict_proba(X_test) # 返回测试集中每个测试样例，分类为每个类的概率
y_prd = pd.DataFrame(y_pro).iloc[:,1]
roc_auc_score(y_test,y_prd)

D:\D(Program)\AI\Anaconda3-4.3.1\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.5393193193193193

In [12]:
# GBDT 重要特征

clf.fit(bb_data,target_bb)
clf_importance = clf.feature_importances_
clf_importance_ = pd.DataFrame(clf_importance)
clf_importance_.columns = {'importance'}
bb_columns = pd.DataFrame(bb_data.columns)
bb_columns.columns={'feature'}

#影响度排序
clf_feature_values = pd.concat([bb_columns,clf_importance_],axis=1)
clf_feature_values = clf_feature_values.sort_values(by='importance')

#影响度非0的特征
clf_feature_well = clf_feature_values[clf_feature_values['importance']!=0]
clf_feature_well_columns = clf_feature_well['feature'].values
clf_feature_well.index = clf_feature_well_columns
columns_GBDT = clf_feature_well.index

In [13]:
columns_GBDT.shape

(50,)

In [14]:
# 测试集提取这些特征，形成新的测试集

C_feature = B_train[columns_GBDT]
new_test = B_test[columns_GBDT]
C_flag = pd.DataFrame(B_train['flag'])
C_train = pd.concat([C_feature,C_flag],axis=1)

CC = C_feature.fillna(0)
new_test_  = new_test.fillna(0)

X_train, X_test, y_train, y_test = cross_validation.train_test_split(CC, C_flag, test_size=0.3, random_state=0)


In [15]:
# XGB模型训练

import xgboost as xgb

xg_train = xgb.DMatrix(X_train,label=y_train)
xg_test = xgb.DMatrix(X_test,label=y_test)


param = {'booster':'gbtree',
         'max_depth':10, 
         'eta':0.1, 
         'silent':1, 
         'objective':'binary:logistic',
         'eval_metric':'auc',
         'subsample': 1,
         "colsample_bytree": 0.7,
         "min_child_weight":2,
              'gamma':3.1,
              'lambda':1,
        "thread":-1,}
num_boost_round = 1500
watchlist = [(xg_train, 'train'), (xg_test, 'eval')]
num_round=15
bst = xgb.train(param, xg_train, num_round)
preds = bst.predict(xg_test)
roc_auc_score(y_test,preds)

0.6038438438438439

In [16]:
xg_train = xgb.DMatrix(CC,label=C_flag)
xg_test = xgb.DMatrix(new_test_)
bst = xgb.train(param, xg_train, num_round)
pro = bst.predict(xg_test)

xgb_model =xgb.cv(param,xg_train,num_boost_round,nfold=5, early_stopping_rounds=300)#, verbose_eval=True

pd.DataFrame(xgb_model)
pd.DataFrame(xgb_model)['test-auc-mean'].mean()

no = pd.DataFrame(NO)

XGB_B=[]
XGB_B=pd.DataFrame(XGB_B)
XGB_B['no'] = no
XGB_B['pred'] = pro

## LGB特征筛选+GBDT训练模型-B数据集-AUC0.587

In [18]:
# LGB的特征工程

import lightgbm as lgb

A_train['label'] = -1
B_train['label'] = 0
B_test['label'] = 1
B_test['flag'] = np.nan

all_data = A_train.append(B_train)
all_data = all_data.append(B_test)
all_data = all_data.reset_index(drop=True)
all_data = all_data.fillna(10)

temp_data = all_data

drop_cols_l = ['flag', 'label', 'no']
train_x = temp_data[temp_data.label==0].drop(drop_cols_l, axis=1)
train_y = temp_data[temp_data.label==0]['flag']

def lgb_feature_selection(tr_x, tr_y, model_seed =666,num_rounds = 500):
    lgb_tr = lgb.Dataset(tr_x, tr_y)
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'random_state': model_seed}
    model = lgb.train(lgb_params, lgb_tr, num_boost_round=num_rounds,verbose_eval=100)
    return model


f_model = lgb_feature_selection(train_x.values, train_y.values)
lgb.plot_importance(f_model, figsize=(16,8))
features_names_im =pd.DataFrame({'feature_name':train_x.columns, 'f_value': f_model.feature_importance()})
features_used = features_names_im[features_names_im.f_value>=0.1*features_names_im.f_value.mean()]



In [19]:
# GBDT训练

tr_x = all_data.loc[all_data.label==0,features_used.feature_name.values].values
test_x = all_data.loc[all_data.label==1,features_used.feature_name.values].values
tr_y = all_data.loc[all_data.label==0, 'flag'].values
test_y = all_data[all_data.label==1][['no']]


gbdt = GradientBoostingClassifier(n_estimators=400, random_state=666)
gbdt.fit(tr_x, tr_y)
gbdt_pred = gbdt.predict_proba(test_x)
gbdt_pred[:, 1]
test_y['pred'] = gbdt_pred[:,1]
GBDT_B = test_y

## GBDT特征筛选+GBDT训练模型-A数据集

In [21]:
A_train = pd.read_csv('data/A_train.csv')
B_train = pd.read_csv('data/B_train.csv')
B_test = pd.read_csv('data/B_test.csv')#//
NO = B_test['no']#//


# In[67]:

B_train_columns = B_train.columns
B_null_count_less = []
B_null_count_large = []

# threshold = 0.63
for i in B_train_columns:
    if ((B_train[i].isnull().sum()) / len(B_train[i]) <= 0.63):
        B_null_count_less.append([i,(B_train[i].isnull().sum()) / len(B_train[i])])
    else:
        B_null_count_large.append([i,(B_train[i].isnull().sum()) / len(B_train[i])])

# len(B_null_count_less) 327

# len(B_null_count_large) 164


B_test_columns = B_test.columns
B_test_count_less = []
B_test_count_large = []

for i in B_test_columns:
    if ((B_test[i].isnull().sum()) / len(B_test[i]) <= 0.63):
        B_test_count_less.append([i,(B_test[i].isnull().sum()) / len(B_test[i])])
    else:
        B_test_count_large.append([i,(B_test[i].isnull().sum()) / len(B_test[i])])
        
        
A_feature = pd.DataFrame(B_null_count_less).values[:,0]
B_feature = pd.DataFrame(B_null_count_less).values[:,0]
BT_feature = pd.DataFrame(B_test_count_less).values[:,0]

a_data = A_train[A_feature]
b_data = B_train[B_feature]
bt_data = B_train[BT_feature]

a_columns = a_data.columns
a_columns = a_columns.sort_values()               #缺失量排序

b_columns = b_data.columns   ## B_train columns，多了一个flag
b_columns = b_columns.sort_values()

bt_columns = bt_data.columns   ## B_test columns
bt_columns = bt_columns.sort_values()

a_data = A_train[a_columns]
b_data = B_train[b_columns]
bt_data = B_test[bt_columns]

b_target = b_data['flag']
a_target = a_data['flag']


b_data.drop('flag',axis=1,inplace=True)
a_data.drop('flag',axis=1,inplace=True)

aa_data = a_data.fillna(1)
bb_data = b_data.fillna(1)
bt_data = bt_data.fillna(1)

bb_data.drop('no',axis=1,inplace=True)
aa_data.drop('no',axis=1,inplace=True)
bt_data.drop('no',axis=1,inplace=True)

from sklearn.ensemble import GradientBoostingClassifier


clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1,  max_depth=2,random_state=0)#
clf.fit(aa_data, a_target)
y_pred = clf.predict_proba(bb_data)
y_pred = pd.DataFrame(y_pred).iloc[:,1]
roc_auc_score(b_target,y_pred)

# clf.fit(aa_data,a_target)
clf_importance = clf.feature_importances_
clf_importance_ = pd.DataFrame(clf_importance)
clf_importance_.columns = {'importance'}
bb_columns = pd.DataFrame(bb_data.columns)
bb_columns.columns={'feature'}

#影响度排序
clf_feature_values = pd.concat([bb_columns,clf_importance_],axis=1)
# feature_values.columns = {'importance','feature'}
clf_feature_values = clf_feature_values.sort_values(by='importance')


#影响度非0的特征
clf_feature_well = clf_feature_values[clf_feature_values['importance']!=0]
clf_feature_well_columns = clf_feature_well['feature'].values
clf_feature_well.index = clf_feature_well_columns

columns_GBDT = clf_feature_well.index


C_feature = A_train[columns_GBDT]
D_feature = B_train[columns_GBDT]
E_feature = B_test[columns_GBDT]


C_flag = pd.DataFrame(A_train['flag'])
D_flag = pd.DataFrame(B_train['flag'])

C_82 = C_feature['UserInfo_82']
C_82 =pd.DataFrame(C_82.fillna(C_feature['UserInfo_82'].median()))
C_82.columns={'new_82'}

D_82 = D_feature['UserInfo_82']
D_82 =pd.DataFrame(D_82.fillna(E_feature['UserInfo_82'].median())) #用B——test的中位数代替有一点提升
D_82.columns={'new_82'}

E_82 = E_feature['UserInfo_82']
E_82 =pd.DataFrame(E_82.fillna(E_feature['UserInfo_82'].median()))
E_82.columns={'new_82'}

newC_feature=pd.DataFrame(C_feature['UserInfo_82']*C_feature['UserInfo_222'])
newC_feature.columns={'new_feature_1'}

newD_feature=pd.DataFrame(D_feature['UserInfo_82']*D_feature['UserInfo_222'])
newD_feature.columns={'new_feature_1'}

newE_feature=pd.DataFrame(E_feature['UserInfo_82']*E_feature['UserInfo_222'])
newE_feature.columns={'new_feature_1'}

C_feature = pd.concat([C_feature,C_82],axis = 1)
D_feature = pd.concat([D_feature,D_82],axis = 1)
E_feature = pd.concat([E_feature,E_82],axis = 1)

C_feature = pd.concat([C_feature,newC_feature],axis = 1)
D_feature = pd.concat([D_feature,newD_feature],axis = 1)
E_feature = pd.concat([E_feature,newE_feature],axis = 1)

C_feature = C_feature.fillna(1)
D_feature = D_feature.fillna(1)
E_feature = E_feature.fillna(1)

from sklearn.ensemble import GradientBoostingClassifier


clf = GradientBoostingClassifier(n_estimators=149, learning_rate=0.66,  max_depth=2, random_state=0,max_features=14,min_weight_fraction_leaf=0.11)
clf.fit(C_feature, C_flag)

y_pred = clf.predict_proba(D_feature)
y_pred = pd.DataFrame(y_pred).iloc[:,1]
roc_auc_score(D_flag,y_pred)




y_pred = clf.predict_proba(E_feature)
y_pred = pd.DataFrame(y_pred).iloc[:,1]
b=pd.DataFrame(y_pred)

no = pd.DataFrame(NO)

A_B_GBDT=[]
A_B_GBDT=pd.DataFrame(A_B_GBDT)
A_B_GBDT['no'] = no
A_B_GBDT['pred'] = y_pred

## LGB特征筛选+LGB训练模型-A数据集

In [22]:
# 特征工程

import lightgbm as lgb

lgb_train = lgb.Dataset(aa_data,label=a_target)
lgb_vd = lgb.Dataset(bb_data,label=b_target)
# lgb_test = lgb.Dataset(D_feature)
# lgb_vd = lgb.Dataset(vd_x, vd_y, reference=lgb_tr)
lgb_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
#     'max_depth':18,
#     'feature_fraction':0.85,
#     'lambda_l1':1.2,
    'random_state': 0}#18     0.85

lgb_model = lgb.train(lgb_params, lgb_train, num_boost_round=200,verbose_eval=True)

lgb_pred = lgb_model.predict(bb_data)

roc_auc_score(b_target,lgb_pred)



lgb_importance = pd.DataFrame(lgb_model.feature_importance(importance_type="split"))
lgb_importance.columns={'importance'}
columns = pd.DataFrame(b_columns).iloc[:-2,:]
columns.columns={'feature'}
lgb_importance = pd.concat([columns,lgb_importance],axis=1)
lgb_importance = lgb_importance.sort_values(by='importance')
lgb_importance = lgb_importance[lgb_importance['importance']>29].reset_index().drop('index',axis=1)
lgb_importance_columns = lgb_importance['feature'].values

C_feature = A_train[lgb_importance_columns]
D_feature = B_train[lgb_importance_columns]
E_feature = B_test[lgb_importance_columns]



C_82 = C_feature['UserInfo_82']
C_82 =pd.DataFrame(C_82.fillna(C_feature['UserInfo_82'].median()))
C_82.columns={'new_82'}

D_82 = D_feature['UserInfo_82']
D_82 =pd.DataFrame(D_82.fillna(E_feature['UserInfo_82'].median())) #用B——test的中位数代替有一点提升
D_82.columns={'new_82'}

E_82 = E_feature['UserInfo_82']
E_82 =pd.DataFrame(E_82.fillna(E_feature['UserInfo_82'].median()))
E_82.columns={'new_82'}

newC_feature=pd.DataFrame(C_feature['UserInfo_253']*C_feature['UserInfo_242'])
newC_feature.columns={'new_feature_1'}

newD_feature=pd.DataFrame(D_feature['UserInfo_253']*D_feature['UserInfo_242'])
newD_feature.columns={'new_feature_1'}

newE_feature=pd.DataFrame(E_feature['UserInfo_253']*E_feature['UserInfo_242'])
newE_feature.columns={'new_feature_1'}

C_feature = pd.concat([C_feature,newC_feature],axis = 1)
D_feature = pd.concat([D_feature,newD_feature],axis = 1)
E_feature = pd.concat([E_feature,newE_feature],axis = 1)

C_feature.drop('UserInfo_134',axis=1,inplace=True)
D_feature.drop('UserInfo_134',axis=1,inplace=True)
E_feature.drop('UserInfo_134',axis=1,inplace=True)


C_feature  = C_feature.fillna(1)
D_feature = D_feature.fillna(1)
E_feature = E_feature.fillna(1)

columns = D_feature.columns





In [23]:
# 模型训练

lgb_params_new = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'max_depth':17,
    'feature_fraction':0.80,
    'lambda_l1':0.6,
#     'scale_pos_weight':1.1,
    'random_state': 0}
                                                   #17 0.80 0.6 1550 0.591  drop134  new82       253X242    1162
lgb_train = lgb.Dataset(C_feature,label=a_target)#17 0.80 0.6 1550 0.589  drop134  new82            1550
lgb_vd = lgb.Dataset(D_feature,label=b_target)   #17 0.80 0.6 1469 0.588 drop134
lgb_model = lgb.train(lgb_params_new, lgb_train, num_boost_round=2000,verbose_eval=500,valid_sets=lgb_vd, early_stopping_rounds=500)#1256

preds = lgb_model.predict(E_feature)
no = pd.DataFrame(NO)

A_B_LGB=[]
A_B_LGB=pd.DataFrame(A_B_LGB)
A_B_LGB['no'] = no
A_B_LGB['pred'] = preds

Training until validation scores don't improve for 500 rounds.
[500]	valid_0's auc: 0.576747
[1000]	valid_0's auc: 0.587195
[1500]	valid_0's auc: 0.59221
[2000]	valid_0's auc: 0.5916
Did not meet early stopping. Best iteration is:
[1608]	valid_0's auc: 0.594327


## 融合

In [26]:
D = pd.DataFrame()
D['no'] = NO
D['pred'] = (A_B_GBDT.iloc[:,1]*0.89 + A_B_LGB.iloc[:,1]*0.113)*0.181+ XGB_B.iloc[:,1]*0.415 + GBDT_B.iloc[:,1].values*0.415
D.to_csv('result.csv',index=False)

# 线上 auc = 0.603206